# ATLAS Test Beam Data

Python script for analysis of ATLAS test beam data. 

The program reads an ASCII (i.e. text) data file, containting a large number of events, where a charged particle (electron or pion) passed through a slice of the ATLAS detector. Each passage is recorded by different detectors (essentially three independent ones!), boiling down to eleven numbers (some more relevant than others). The exercise is to separate electron and pion events based on these numbers, and in turn use this information to measure the interaction (i.e. detector performance) of pions and electrons seperately.

NOTE 1: In this exercise, don't consider (at least at first) the details, but just rely on the advice from particle physicist, that in these detectors, electrons tend to give a higher signal (i.e. values) than pions do, and that none of the particle types are very rare.

NOTE 2: Though the data is from particle physics, it could in principle have been from ANY other source, and the eleven numbers could for example have been indicators of cancer, key numbers for investors, or index numbers for identifying potential costumors. Thus the advice from above could be from and old doctor, economist, or marketing consultant.

NOTE 3: This is real data, and while it has been prepared somewhat, it might contain surprises and does not necessarily follow known PDFs.

For more information on ATLAS test beam: http://www.nbi.dk/~petersen/Teaching/Stat2025/TestBeam/TestBeamDataAnalysis.html.

***

### Authors: 
- Troels C. Petersen (Niels Bohr Institute)


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from iminuit import Minuit, cost
from scipy import stats

Set parameters of the program:

In [ ]:
# Write extensive output
verbose = True
N_verbose = 20

Open file, `DataSet_AtlasPid_ElectronPion_2GeV.txt`, and read in all the data:

In [ ]:
data = np.loadtxt('DataSet_AtlasPid_ElectronPion_2GeV.txt', skiprows=1, unpack=True)
Cher, nLT, nHT, EM0, EM1, EM2, EM3, Had0, Had1, Had2, Muon = data
Cher.shape

# It is always very "healthy" to inspect the data visually/manually, just to get a quick feel for it.
if (verbose) :
    for i in range (N_verbose) :
        print(f"Cher: {Cher[i]:6.1f}    nLT, nHT: {int(nLT[i]):2d}, {int(nHT[i]):2d}    EM: {EM0[i]:5.2f} {EM1[i]:5.2f} {EM2[i]:5.2f} {EM3[i]:5.2f}   Had: {Had0[i]:5.2f} {Had1[i]:5.2f} {Had2[i]:5.2f}    Muon: {Muon[i]:5.1f}")

## Your analysis:

Through plotting and making selections in one/several variable(s) seeing the impact (e.g. how many events remain) in others, work out what this sample consists of. 

An example plot could be to see if there are any clear types difference between the entries in the data, and if this gives rise to a way of selecting 

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(16, 10))

ax[0,0].hist(Cher, bins=200, range=(400, 1400), histtype='step', label='Cherenkov')
ax[0,0].set(xlabel="Cherenkov", ylabel="Frequency")

ax[1,1].hist(EM0,  bins=200, range=(-0.5, 2.0), histtype='step', label='EM0')
ax[1,1].hist(EM1,  bins=200, range=(-0.5, 2.0), histtype='step', label='EM1')
ax[1,1].hist(EM2,  bins=200, range=(-0.5, 2.0), histtype='step', label='EM2')
ax[1,1].hist(EM3,  bins=200, range=(-0.5, 2.0), histtype='step', label='EM3')
ax[1,1].set(xlabel="Energy in ElectroMagnetic Calorimeter Layer 0-3 (EM0-3)", ylabel="Frequency")

h = ax[0,1].hist2d(EM1,  Cher, bins=(100, 100), range=((-0.5, 2.0), (400, 1400)), norm=mpl.colors.LogNorm(), cmap="Reds")
plt.colorbar(h[3], ax=ax[0, 1])                    # z-scale on the right of the figure
ax[0,1].set(xlabel="EM1", ylabel="Cherenkov")

# You can add your own fourth plot:
fig.tight_layout()

NOTE: The simple plotting function gives a warning for EM1 (not Cher). Ask yourself, if you did a check of the original data file input values in any way? That is always "healthy"!
***

## Questions to be answered:

Generally, this analysis is about separating electrons and pions (and determining how well this can be done), followed by a few questions characterizing the detector response to each type of particle. Thus, you should imagine, that your new detector/equipment/questionaire gave you this output, and now it is up to you to find out, what this tells you about your experiment, and how to extract information from it in the best possible way. Typically, it will have taken you months (if not years) to get to this point.

Note that this data is NOT meant for “fine tuned analysis”, but rather “crude inspection”. You should try to get simple approximate answers out - I’m sure that you will afterwards be able to fine tune them.

Below are questions guiding you, some/most of which your analysis should cover, but you do **not** have to follow them blindly (I've put "Optional" on those that are not essential). Start by considering the data, and get a feel for the typical range of each variable. Plot the variables, both in 1D and also 2D! From considering these plots, guess/estimate an approximate knowledge of how electrons and pions distribute themselves in the variables above, and how to make a selection of these.

As described on the webpage introducing the data, there are three (relevant) detectors:
- Cherenkov (1 value),
- TRT (Transition Radiation Tracker, 2 values) and
- Calorimeters (4 values)

They are each capable of separating electrons and pions. As they are (largely) _INDEPENDENT_ (three separate detectors), they may be used to cross check each other, and this is what you should use, in fact the essential part of this (and many other) analysis!


Questions:
----------
1. Find for each of these three detector systems one variable, which seem to separate electrons and pions best. For example, start with the Cherenkov, which is only a single number, and assume/guess that the large peak at low values is mainly from pions, while the upper broad peak is from electrons (this you would know, as you designed the experiment). Now plot the TRT and Calorimeter distributions when the Cherenkov selects a pion and afterwards an electron. This should give you a good idea about how to separate pions and electrons using the TRT and Calorimeters.

    Hint: Sometimes variables from a single detector are more powerful, when they are combined, e.g. taken ratios of (or used in a Fisher or ML algorithm). For the TRT this may be somewhat doable, but for the EMcalo, it is not as simple. Here, one variable caries most of the separation power, but involving other layers may enhance the separation power. However, to begin with, just consider a single number from each detector.


2. Next you should try to see, if you can make a selection, which gives you a fairly large and clean electron and pion sample, respectively. The question is, how can you know how clean your sample is and how efficient your selection is?  This can actually be measured in the data itself, using the fact that there are three independent detectors. For example, start by making an electron and a pion selection using two of the three variables, and plot the third variable for each of these selections. Now you can directly see, how electrons and pions will distribute themselves in this third variable. Are you worried, that there are pions in your electron sample, and vice versa? Well, there will probably be, but so few, that it won't matter too much, at least not to begin with. Why? Well, let us assume that for each detector, 80% of electrons pass your requirement, but also 10% of pions do. Assuming an even number of electrons and pions (which is not really the case), then with two detector cuts, you should get a sample, which is: $\frac{0.8\cdot0.8} {0.8\cdot0.8 + 0.1\cdot0.1} = 98.5\%$ pure.

    Now with this sample based on cuts on the two other detectors, ask what fraction of electrons and pions passes your electron selection in the remaining detector. The fraction of electrons, that are not selected as electrons will be your TYPE I errors (False Negative Rate), denoted alpha, while the fraction of pions, that do pass the cut will be your TYPE II errors (False Positive Rate), denoted beta. Measure these for each of the two cuts in the three detector types, and ask yourself if they are "reasonable", i.e. something like in the example above. If not, then you should perhaps reconsider adjusting your cuts.

By now, you should for each detector have 6 numbers (first considering electrons and then pions as "Positive"):
    - The electron cut value above which you accept an electron.
    - The efficiency (i.e. True Positive Rate) for electrons of this cut.
    - The fake rate (i.e. False Positive Rate) for pions of this cut.
    - The pion cut value below which you accept a pion (may be same value as above for electrons!).
    - The efficiency (i.e. True Positive Rate) for pions of this cut.
    - The fake rate (i.e. False Positive Rate) for electrons of this cut.


3. Given the efficiencies and fake rates of each cut, try to combine these (again assuming that they are independent) into knowledge of your sample purities and also the total number of electrons and pions in the whole sample. Do the sum of estimated electrons and pions added actually match the number of particles in total? This is a good cross check!

### More advanced questions:

4. If the number of pions was suddenly 1000 times that of elections, would you still be able to get a sample of fairly pure (say 90% pure) electrons? And if so, what would the efficiency for these electrons be? That is roughly equivalent of asking, if you can get a 99.9% pure electron sample from the data given (where pions do not dominate in numbers).


5. Expanding on problem 2), try now to calculate ROC curves for each of the three detectors. These are obtained by making a clean selection using the two other detectors for electrons and pions seperately, and then integrating over these two distributions, using the running (normalised) integral of each as x and y coordinate in a (ROC) curve. If you do not manage on your own, perhaps consider the ROC calculator example, which is posted along with this exercise.


6. One of the purposes of the testbeam was to measure the response of the TRT detector to exactly electrons and pions. Consider for example only events that has 33 TRT hits (i.e. `nLT` $= 33$). As the High-Threshold probability (i.e. probability of passing the High-Threshold, given that the Low-Threshold was passed), is assumed to be constant in the TRT detector (but quite different for electrons and pions), what distribution should the number of High-Threshold hits (`nHT`) follow? And is that really the case, both for electrons and pions?


7. Still considering `nLT` $=33$, and given that there are both electrons and pions in the sample (each with a different HT probability), `nHT` should in the unselected data be a combination of two distributions. Try to fit the number of HT hits with two distributions combined. Do they fit the data? And can this fit be used to estimate the fraction of pions and electrons in the sample? And does that estimate match you previous estimate? Perhaps retry with other values for the number of TRT hits.


8. Try to select pions using three different (mutually exclusive) techniques:
    1. Passing only a hadronic calorimeter requirement (e.g. that the sum of the three HCal values is above some minimum energy).
    2. Passing only Cherenkov AND EMcalo requirements.
    3. Passing both A) and B).

Try to measure the HT probability (i.e. fraction of High-Threshold hits) for each of these three pion samples. Do they agree with each other?

# Learning points:

This exercise might be the closest thing to reality regarding real data that is not easily modelled. Note that the original raw data is a lot more messy, and that a lot of cleaning has taken place, before you got it!

From this exercise you should realise:
1. that real data is not as clean-cut, well-behaved, and Gaussian as you might think.
2. that the use of **independent variables is very powerful** in data analysis.
3. that you need to be sharp regarding TPR, FPR, TNR, and FNR... :-)
4. that the sign of a good analysis is, that cross checks have been thought into it (e.g. end of Q.3)

## Example solution

As our data is unlabeled, we first need to establish labels, so we later on can determine other detectors efficiacy of distinhushing between electrons and pions. Here we can use our knowledge about electrons and pions and their expected mesaurement in the cherikov and EM1 detector, and try to "cut out" the pions as seen below. This mask is not perfect, but it should label most of the electrons and pions correct. We check this later

In blue the pions are marked, and in orange the electrons

In [ ]:
Pions = (Cher < 640) * (EM1 < 0.35)
electrons = ~Pions # We make labels

plt.scatter(EM1[Pions], Cher[Pions], alpha = 0.1)
plt.scatter(EM1[electrons], Cher[electrons], alpha = 0.1)

### Comparing different detectors
As we now have a labeling, we can use this to compute the TPR and FPR, of the different detectors on their own. Below is shown 6 histograms for the first detectors, with a cutoff shown to distinguish electrons and pions, given the distributions.
Looking at the results, many of the detectros give a poor clasification, bordering to a random classifier, but two detectors seem very promissing, EM1 and nHT. Here both of the detctors measure a high TPR, while having a low FPR.

On the bottom plot we have shown the distribution of the hadron measurement. Electrons are not expected to reach this, making it a bad detector to classify between the two, but we can use it as a type of sanity to check to see that our initial labeling was good, as we then only would expect pions represented in the plots. Looking at the plots a majority of the particles reached is pions, while only a few "electrons" made it thorugh. Computing the fraction of the labeled pions that made it through relative to the total amount of particles that made it thorugh we end up with a fraction in the high 80 and 90, suggesting that the cut we made in the beginning was great at labeling pions.

In [ ]:
fig, ax = plt.subplots(1,5, figsize = (15, 3))
ax[0].hist(nLT[Pions], bins = np.arange(100), histtype = 'step')
ax[0].hist(nLT[electrons], bins = np.arange(100), histtype = 'step')
ax[0].set_title('nLT')
ax[0].vlines(37, 0, 2000, color = 'tab:red', ls = '--')
tp = sum(nLT[Pions] < 37)
fp = sum(nLT[electrons] < 37)
fn = sum(nLT[Pions] > 37)
tn = sum(nLT[electrons] > 37)
FPR = fp/(fp + tn)
TPR = tp/(tp + fn)
ax[0].text(0.1, 0.9, f"TPR = {TPR:3.5f} \nFPR = {FPR:3.5f}", family='monospace', fontsize=7, transform = ax[0].transAxes)

ax[1].hist(nHT[Pions], bins = np.arange(40), histtype = 'step')
ax[1].hist(nHT[electrons], bins = np.arange(40), histtype = 'step')
ax[1].set_title('nHT')
ax[1].vlines(3, 0, 4000, color = 'tab:red', ls = '--')
tp = sum(nHT[Pions] < 3)
fp = sum(nHT[electrons] < 3)
fn = sum(nHT[Pions] > 3)
tn = sum(nHT[electrons] > 3)
FPR = fp/(fp + tn)
TPR = tp/(tp + fn)
ax[1].text(0.1, 0.9, f"TPR = {TPR:3.5f} \nFPR = {FPR:3.5f}", family='monospace', fontsize=7, transform = ax[1].transAxes)


ax[2].hist(EM1[Pions], bins = np.linspace(-0.5, 2.0, 100), histtype = 'step')
ax[2].hist(EM1[electrons], bins = np.linspace(-0.5, 2.0, 100), histtype = 'step')
ax[2].set_title('EM1')
ax[2].vlines(0.3, 0, 1200, color = 'tab:red', ls = '--')
tp = sum(EM1[Pions] < 0.3)
fp = sum(EM1[electrons] < 0.3)
fn = sum(EM1[Pions] > 0.3)
tn = sum(EM1[electrons] > 0.3)
FPR = fp/(fp + tn)
TPR = tp/(tp + fn)
ax[2].text(0.1, 0.9, f"TPR = {TPR:3.5f} \nFPR = {FPR:3.5f}", family='monospace', fontsize=7, transform = ax[2].transAxes)


ax[3].hist(EM2[Pions], bins = np.linspace(-0.5, 2.0, 100), histtype = 'step')
ax[3].hist(EM2[electrons], bins = np.linspace(-0.5, 2.0, 100), histtype = 'step')
ax[3].set_title('EM2')
ax[3].vlines(0.33, 0, 1200, color = 'tab:red', ls = '--')
tp = sum(EM2[Pions] < 0.33)
fp = sum(EM2[electrons] < 0.33)
fn = sum(EM2[Pions] > 0.33)
tn = sum(EM2[electrons] > 0.33)
FPR = fp/(fp + tn)
TPR = tp/(tp + fn)
ax[3].text(0.1, 0.9, f"TPR = {TPR:3.5f} \nFPR = {FPR:3.5f}", family='monospace', fontsize=7, transform = ax[3].transAxes)


ax[4].hist(EM3[Pions], bins = np.linspace(-0.5, 2.0, 100), histtype = 'step')
ax[4].hist(EM3[electrons], bins = np.linspace(-0.5, 2.0, 100), histtype = 'step')
ax[4].set_title('EM3')
ax[4].vlines(0.03, 0, 5000, color = 'tab:red', ls = '--')
tp = sum(EM3[Pions] < 0.03)
fp = sum(EM3[electrons] < 0.03)
fn = sum(EM3[Pions] > 0.03)
tn = sum(EM3[electrons] > 0.03)
FPR = fp/(fp + tn)
TPR = tp/(tp + fn)
ax[4].text(0.1, 0.9, f"TPR = {TPR:3.5f} \nFPR = {FPR:3.5f}", family='monospace', fontsize=7, transform = ax[4].transAxes)



fig, ax = plt.subplots(1,3, figsize = (15, 3))

ax[0].hist(Had0[Pions], bins = np.linspace(0.01,2, 50), histtype = 'step')
ax[0].hist(Had0[electrons], bins = np.linspace(0.01,2, 50), histtype = 'step')
ax[0].set_title('Had0')
freq_pions, _ = np.histogram(Had0[Pions], bins = np.linspace(0.01, 2, 50))
freq_electrons, _ = np.histogram(Had0[electrons], bins = np.linspace(0.01, 2, 50))
print(freq_pions.sum()/(freq_pions.sum() + freq_electrons.sum()))



ax[1].hist(Had1[Pions], bins = np.linspace(0.01,2, 50), histtype = 'step')
ax[1].hist(Had1[electrons], bins = np.linspace(0.01,2, 50), histtype = 'step')
ax[1].set_title('Had1')
freq_pions, _ = np.histogram(Had1[Pions], bins = np.linspace(0.01, 2, 50))
freq_electrons, _ = np.histogram(Had1[electrons], bins = np.linspace(0.01, 2, 50))
print(freq_pions.sum()/(freq_pions.sum() + freq_electrons.sum()))

ax[2].hist(Had2[Pions], bins = np.linspace(0.01,2, 50), histtype = 'step')
ax[2].hist(Had2[electrons], bins = np.linspace(0.01,2, 50), histtype = 'step')
ax[2].set_title('Had2')
freq_pions, _ = np.histogram(Had2[Pions], bins = np.linspace(0.01, 2, 50))
freq_electrons, _ = np.histogram(Had2[electrons], bins = np.linspace(0.01, 2, 50))
print(freq_pions.sum()/(freq_pions.sum() + freq_electrons.sum()))

plt.show()

### Combining measurements
We have seen that individual detectors can make a great selection on their own, but now we want to investigate if we can combine them to create an even beter destinction. To do this we use a LDA, where we can include and exclude different detectors as we choose. In the end we compute a roc curve, making the comparison between them better.

In [ ]:
def LDA(data, include, pmask, emask):
    data = data[include,:]
    pions = data[:, pmask]
    electrons = data[:,emask]
    pions_nanmask = np.ones(sum(pmask))
    electrons_nanmask = np.ones(sum(emask))
    for i in range(len(include)):
        pions_nanmask *= ~np.isnan(pions[i,:])
        electrons_nanmask *= ~np.isnan(electrons[i,:])
    pions = pions[:, pions_nanmask.astype(bool)]
    electrons = electrons[:, electrons_nanmask.astype(bool)]

    mean_pions = np.mean(pions, axis = 1)
    mean_electrons = np.mean(electrons, axis = 1)

    cov_pions = np.cov(pions)
    cov_electrons = np.cov(electrons)
    wf = np.linalg.inv(cov_pions + cov_electrons) @ (mean_pions - mean_electrons)
    pions_proj = pions.T @ wf
    electrons_proj = electrons.T @ wf
    return pions_proj, electrons_proj, wf

def compute_rates(TA, TB, thr):
    tp = sum(TB < thr)
    fp = sum(TA < thr)
    fn= sum(TB > thr)
    tn = sum(TA > thr)


    
    FPR = fp/(fp + tn)
    TPR = tp/(tp + fn)
    return TPR, FPR

def calc_ROC(hist1, hist2) :

    # First we extract the entries (y values) and the edges of the histograms:
    # Note how the "_" is simply used for the rest of what e.g. "hist1" returns (not really of our interest)
    y_sig, x_sig_edges = hist1 
    y_bkg, x_bkg_edges = hist2
    
    # Check that the two histograms have the same x edges:
    if np.array_equal(x_sig_edges, x_bkg_edges) :
        
        # Extract the center positions (x values) of the bins (both signal or background works - equal binning)
        x_centers = 0.5*(x_sig_edges[1:] + x_sig_edges[:-1])
        
        # Calculate the integral (sum) of the signal and background:
        integral_sig = y_sig.sum()
        integral_bkg = y_bkg.sum()
    
        # Initialize empty arrays for the True Positive Rate (TPR) and the False Positive Rate (FPR):
        TPR = np.zeros_like(y_sig) # True positive rate (sensitivity)
        FPR = np.zeros_like(y_sig) # False positive rate ()
        
        # Loop over all bins (x_centers) of the histograms and calculate TN, FP, FN, TP, FPR, and TPR for each bin:
        for i, x in enumerate(x_centers): 
            
            # The cut mask
            cut = (x_centers < x)
            
            # True positive
            TP = np.sum(y_sig[~cut]) / integral_sig    # True positives
            FN = np.sum(y_sig[cut]) / integral_sig     # False negatives
            TPR[i] = TP / (TP + FN)                    # True positive rate
            # print(TP, FN, TPR[i])
            
            # True negative
            TN = np.sum(y_bkg[cut]) / integral_bkg      # True negatives (background)
            FP = np.sum(y_bkg[~cut]) / integral_bkg     # False positives
            FPR[i] = FP / (FP + TN)                     # False positive rate            
        return FPR, TPR
    
    else:
        AssertionError("Signal and Background histograms have different bins and/or ranges")



def ROC(data, include, pmask, emask):
    pionsLDA, electronsLDA, wf = LDA(data, include, Pions, electrons)

    PionsHist = np.histogram(pionsLDA, np.linspace(-40,10,100))
    ElectronsHist = np.histogram(electronsLDA, np.linspace(-40,10,100))
    return calc_ROC(PionsHist, ElectronsHist)



In [ ]:
res1 = ROC(data, [1,2,3], Pions, electrons)
res2 = ROC(data, [1,2,3,4,5,6], Pions, electrons)
res3 = ROC(data, [1,2,3,8,9,10], Pions, electrons)
# print(res2[0])

plt.plot(res1[0], res1[1])
plt.plot(res2[0], res2[1])
plt.plot(res3[0], res3[1])
plt.show()